1) Создать скрипт для анализа тестовых файлов - сбора статистики упоминания слов. Аргументом командной строки является имя файла, который нужно проанализировать. Второй аргумент - количество слов, которые нужно выводить на экран (если аргумент не указан, то выводим статистику по 100 словам, статистика выводится в порядке убывания частоты слов, по каждому слову на экран выводится строка в формате "<слово> <частота>"). Если второй аргумент не целое число, то его рассматриваем как имя файла, в который в формате CSV с заголовком сохраняем статистику слов (Первый столбец - слово, второй столбце - частота).

In [ ]:
import sys
import string
import csv
from collections import Counter

def analyze_file(file_name, count=100):
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            text_str = file.read().lower()  # убираем заглавные буквы
            translator = str.maketrans({char: '' for char in string.punctuation})  # правила преобразования строки
            # .maketrans(dict) или .maketrans(char, to_char) или .maketrans(arg1, arg2, arg3)
            text_str = text_str.translate(translator)  # преобразование строки - убираем знаки пунктуации
            text_words = text_str.split()

            # класс collections.Counter() предназначен для удобных и быстрых подсчетов 
            # количества появлений неизменяемых элементов в последовательностях.
            # A Counter is a dict subclass for counting hashable objects.
            word_freq = Counter(text_words)  # имеет вид Counter({'str1': number1, 'str2': number2, ...})
            sorted_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)  # сортируем по количеству вхождений слова в тексте

            if isinstance(count, int):  # пренадлежит ли count к типу int
                output_words = sorted_freq[:count]
                for word, freq in output_words:
                    print(f"{word} {freq}")
            else:
                try:
                    with open(count, mode='w', newline='') as file_csv:  # если нам нужно создать csv файл с именем count
                        writer = csv.writer(file_csv)
                        writer.writerow(['слово', 'частота'])
                        writer.writerows(sorted_freq)
                except:
                    print(f'нельзя сохранить в файл \'{count}\' или ошибка при сохранении в \'{count}\'')
                    return
    except:
        print(f'\n ~ ошибка при считывании из \'{file_name}\' или файл \'{file_name}\' нельзя открыть ~ \n')
        return


if name == "main":
    # argv - список аргументов командной строки
    # argv[0] - имя скрипта - всегда и везде
    # argv[1] - имя файла откуда берем инфу
    # argv[2] - если есть, то количество строк/ название файла куда сохранить информацию
    
    if len(sys.argv) < 2:
        print('\n ~ неправильные аргументы ~ \n')
        sys.exit()  # выход из Python, поднятие исключения SystemExit (искл можно перехватить)
    elif len(sys.argv) == 3:
        try:
            count = int(sys.argv[2])
        except ValueError:
            count = sys.argv[2]
        analyze_file(sys.argv[1], count)
    else:
        analyze_file(sys.argv[1])

2) Создать скрипт, которому в командной строке передаются имена файлов. В файлах в формате CSV с заголовком содержится информация о частоте слов: первый столбец - слово, второй столбце - частота. Выполнить агрегацию информации о частоте слов из всех переданных файлов (т.е. рассчитать суммарную частоту для всех упомянутых слов). Результат сохранить в файл с именем, которое введет пользователь в ответ на соответствующий запрос.
2.1) Скрипт должен адекватно реагировать на ситуацию отсутствия файлов и другие проблемы с вводом/выводом.
2.2) Оформить модуль с функциями, которые используются в задании 1 и 2.

In [ ]:
def analyze_file(file_name, count=100):
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            text_str = file.read().lower()  # убираем заглавные буквы
            translator = str.maketrans({char: '' for char in string.punctuation})  # правила преобразования строки
            # .maketrans(dict) или .maketrans(char, to_char) или .maketrans(arg1, arg2, arg3)
            text_str = text_str.translate(translator)  # преобразование строки - убираем знаки пунктуации
            text_words = text_str.split()

            # класс collections.Counter() предназначен для удобных и быстрых подсчетов 
            # количества появлений неизменяемых элементов в последовательностях.
            # A Counter is a dict subclass for counting hashable objects.
            word_freq = Counter(text_words)  # имеет вид Counter({'str1': number1, 'str2': number2, ...})
            sorted_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)  # сортируем по количеству вхождений слова в тексте

            if isinstance(count, int):  # пренадлежит ли count к типу int
                output_words = sorted_freq[:count]
                for word, freq in output_words:
                    print(f"{word} {freq}")
            else:
                try:
                    with open(count, mode='w', newline='') as file_csv:  # если нам нужно создать csv файл с именем count
                        writer = csv.writer(file_csv)
                        writer.writerow(['слово', 'частота'])
                        writer.writerows(sorted_freq)
                except:
                    print(f'нельзя сохранить в файл \'{count}\' или ошибка при сохранении в \'{count}\'')
                    return
    except:
        print(f'\n ~ ошибка при считывании из \'{file_name}\' или файл \'{file_name}\' нельзя открыть ~ \n')
        return


def aggregate_files(file_names):
    word_freq = {}
    for file_name in file_names:
        # в словарь word_freq добавляем новые слова или обновляем частоту для каждого файла из списка файлов 
        try:
            with open(file_name, 'r') as file:
                csv_reader = csv.reader(file)
                next(csv_reader)  # пропускаем первую строку - заголовок
                for row in csv_reader:
                    word, freq = row[0].split(';')  # разделяем ';' т.к. такой разделитель в моих файлах
                    word_freq[word] = word_freq.get(word, 0) + int(freq)  # добавляем частоту слов в данном файле к общей сумме
                                                                          # .get(ключ, значение default = 0)    ->    {'ключ': 0}
        except:
            print(f'\n ~ ошибка при считывании из \'{file_name}\' или файл \'{file_name}\' нельзя открыть ~ \n')
            return
    output_file = input('имя файла для сохранения результата: ')
    # сохраняем  в указанный файл
    try:
        with open(output_file, 'w', newline='') as file:
            csv_writer = csv.writer(file, delimiter=';')  # ра зделяем ';' т.к. такой формат поймет excel
            csv_writer.writerow(['слово', 'суммарная частота'])  # добавляем заголовок
            for word, freq in word_freq.items():
                csv_writer.writerow([word, freq])  # добавляем элементы
        print(f'результат сохранен в {output_file}')
    except:
        print(f'нельзя сохранить в файл \'{output_file}\' или ошибка при сохранении в \'{output_file}\'')
        return


if name == "main":
    # argv - список аргументов командной строки
    
    if len(sys.argv) < 2:
        # если ни один файл для считывания не указан
        print('\n ~ неправильные аргументы ~ \n')
        sys.exit()  # выход из Python, поднятие исключения SystemExit (искл можно перехватить)
    aggregate_files(sys.argv[1:])

3.1) Написать программу, которая запускается из консоли и считает сумму переданных в неё чисел
3.2) Добавить в эту программу произведение
3.3) По переданному параметру считать либо сумму, либо произведение
3.4) Оформить это дополнительный модуль.

In [ ]:
import sys


def sum_of_n(numbers):
    # сумма чисел
    sum_of_numbers = 0.0
    for numb in numbers:
        try:
            sum_of_numbers += float(numb)
        except:
            print('\n ~ не все аргументы - числа! ~ \n')
            return
    return sum_of_numbers


def mult_of_n(numbers):
    # произведение чисел
    mult_of_numbers = 1
    for numb in numbers:
        try:
            mult_of_numbers *= float(numb)
        except:
            print('\n ~ не все аргументы - числа! ~ \n')
            return
    return mult_of_numbers


if name == 'main':
    if len(sys.argv) < 2:
        # если ни одно число не указано
        print('\n ~ неправильные аргументы ~ \n')
        sys.exit()  # выход из Python, поднятие исключения SystemExit (искл можно перехватить)
        
    # argv[1] - выбираем тип действия
    match sys.argv[1]:
        case 'sum':
            print(sum_of_n(sys.argv[2:]))
        case 'mult':
            print(mult_of_n(sys.argv[2:]))
        case _:
            print('\n ~ вы не выбрали ни умножение, ни сумму')

4) Реализовать модуль calculate.py, содержащий набор функций для работы с целыми числами, записанными как на русском языке (например: тридцать восемь), так и в виде простых арифметических выражений (например: 2 + 3).
На базе этого модуля подготовить скрипт calculate.py со следующими возможностями:
1) python claculate.py to_numbers my_file.txt
Вызов приводит к преобразованию всех чисел, написанных на русском языке в числовом виде и сохранению изменений в файл с тем же именем.
python claculate.py to_numbers my_file.txt my_file_res.txt
(результат сохраняется в файл с именем my_file_res.txt)

In [ ]:
import words2numsrus
import re


def evaluate_arithm_exp(text):
    pattern = r'\b(\d+)\s*([-+*/])\s*(\d+)\b'

    def replace_expr(match):
        return str(eval(match.group(0)))

    evaluated_text = re.sub(pattern, replace_expr, text)
    return evaluated_text


def to_numbers(argv_1, input_file, output_file=None):
    with open(input_file, 'r', encoding='utf-8') as f:
        text = f.read()
    converted_text = ''
    if argv_1 == 'rus':
        extractor = words2numsrus.NumberExtractor()
        converted_text = extractor.replace_groups(text)
    elif argv_1 == 'math':
        converted_text = evaluate_arithm_exp(text)
    else:
        print(' некорректный аргумент 1 ')
        return
    
    if not output_file:
        output_file = input_file
        
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(str(converted_text))
        

if name == "main":
    if len(sys.argv) < 3:
        print("\n неправильное количество аргументов \n")
        sys.exit()
    
    input_file = sys.argv[2]
    output_file = sys.argv[3] if len(sys.argv) == 4 else sys.argv[2]
    
    to_numbers(argv[1], input_file, output_file)